In [ ]:
# notebook for first canonical problem for multimodal optimization looking for localized extreme
# events with agent-based military simulation
# Experiment 2.1 - Deterministic literature test problem "uneven decreasing maxima" [5]
# Part 1 - balanced strategy (PIONEER)
#
# Author: Alex Braafladt
# Initial creation: 3/11/23
#
# Goal: Benchmark state-of-the-art Bayesian Optimization approaches (and QMC/MCS) on a multimodal
#       optimization test function focused on localized extrema
#
# Notes:
# -Using the BoTorch framework [1] and NEI from [4]
# -TEAD technique from [2]
# -TuRBO technique from [3]
#
# References:
# [1] M. Balandat et al., “BOTORCH: A framework for efficient Monte-Carlo Bayesian optimization,”
#     Adv. Neural Inf. Process. Syst., vol. 2020-Decem, no. MC, 2020.
# [2] S. Mo et al., “A Taylor Expansion-Based Adaptive Design Strategy for Global Surrogate
#     Modeling With Applications in Groundwater Modeling,” Water Resour. Res., vol. 53, no.
#     12, pp. 10802–10823, 2017, doi: 10.1002/2017WR021622.
# [3] D. Eriksson, M. Pearce, J. R. Gardner, R. Turner, and M. Poloczek, “Scalable global
#     optimization via local Bayesian optimization,” Adv. Neural Inf. Process. Syst., vol.
#     32, no. NeurIPS, 2019.
# [4] B. Letham, B. Karrer, G. Ottoni, and E. Bakshy, “Constrained Bayesian optimization with noisy
#     experiments,” Bayesian Anal., vol. 14, no. 2, pp. 495–519, 2019, doi: 10.1214/18-BA1110.
# [5] X. Li, A. Engelbrecht, and M. G. Epitropakis, “Benchmark Functions for CEC’2013 Special
#     Session and Competition on Niching Methods for Multimodal Function Optimization,” 2016.

In [ ]:
# imports
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from extremasearch.globalmm.globalsearch import MultimodalExtremaSearch
from botorch import fit_gpytorch_mll
from botorch.models.gp_regression import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import networkx as nx
from botorch.models.transforms import Normalize, Standardize
import time


# setup
dtype = torch.double
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
# setup file i/o
import os as os
import datetime as dt
# get current working directory
wrkdir = os.getcwd()
print('Current working directory: '+wrkdir)
# set up a data save directory for all future runs
newoutputdir = wrkdir+'\output'
if not os.path.exists(newoutputdir):
    os.makedirs(newoutputdir)
# set up a new directory to store files for the current run - updates at each new full run of notebook
curDatetime = dt.datetime.now()
datasavedir = newoutputdir + r'\\' + 'exp2.1_balanced_lit2' + str(curDatetime.strftime('%Y%m%d%H%M%S'))
if not os.path.exists(datasavedir):
    os.makedirs(datasavedir)
print('Data save directory: '+datasavedir)

In [ ]:
# test function
def udm_torch(x):
    x = x
    y = torch.zeros(x.shape)
    y += torch.exp(-2.0*torch.log(torch.tensor(2.0))*((x - 0.08)/0.854)**2.0) * torch.sin(5.0*torch.pi*(x**(3.0/4.0)-0.05))**6.0
    return y

In [ ]:
# plot the test function
x1 = torch.linspace(0,1.0,200)
y1 = udm_torch(x1)
fig, ax = plt.subplots(1, 1, figsize=(8,6))
sns.lineplot(x=x1, y=y1, ax=ax, color='r')
# ax.set_xlim([0.9, 1.0])
# ax.set_ylim([0.2, 0.35])
plt.savefig(datasavedir + '/'+'deterministic_test_problem'+'.png')

In [ ]:
# wrapper for use in optimization loop
def outcome_objective(x):
    """wrapper for the outcome objective function"""
    return udm_torch(x).type_as(x)

In [ ]:
# functions for getting and working with data from searches
def get_bounds(graph):
    """Get the bounds for each node in a graph to be able to plot them"""
    graph_leaves = [n for n in graph if graph.out_degree[n] == 0]
    bound_list = []
    node_list = []
    for n in graph_leaves:
        # get current leaf node
        current_node = graph.nodes()[n]
        current_state = current_node['data']
        current_bounds = current_state.local_bounds
        node_list.append(n)
        bound_list.append(current_bounds)
    return node_list, bound_list


def get_bounds_all(graph):
    """Get the bounds for each node in a graph to be able to plot them"""
    graph_nodes = [n for n in graph]
    bound_list = []
    node_list = []
    for n in graph_nodes:
        # get current node
        current_node = graph.nodes()[n]
        current_state = current_node['data']
        current_bounds = current_state.local_bounds
        node_list.append(n)
        bound_list.append(current_bounds)
    return node_list, bound_list


def fit_local_models(graph):
    """Go through leaf nodes and update LocalSearchState to have local {x, y}
    based on global {x, y} that are within bounds"""
    graph_leaves = [n for n in graph if graph.out_degree[n] == 0]
    for n in graph_leaves:
        current_node = graph.nodes()[n]
        current_state = current_node['data']
        current_state.local_model = SingleTaskGP(current_state.x_local, current_state.y_local,
                                                 input_transform=Normalize(d=current_state.x_local.shape[-1]),
                                                 outcome_transform=Standardize(m=current_state.y_local.shape[-1]))
        current_state.local_mll = ExactMarginalLogLikelihood(current_state.local_model.likelihood,
                                                             current_state.local_model)
        fit_gpytorch_mll(current_state.local_mll)


def plot_local_models_1d(graph, x_search, y_search, rep):
    """Go through leaf nodes and plot the local models only within the bounds of the node"""
    fit_local_models(graph)
    nodes, bnds = get_bounds(graph)
    x_test = torch.linspace(0, 1, 400)
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    # true objective
    ax.plot(x_test.numpy(), outcome_objective(x_test).numpy(), 'r-', alpha=0.9, label='true objective')
    # local models
    graph_leaves = [n for n in graph if graph.out_degree[n] == 0]
    for n in graph_leaves:
        current_node = graph.nodes()[n]
        current_state = current_node['data']
        current_model = current_state.local_model
        current_bounds = current_state.local_bounds
        current_x_test_mask = torch.ge(x_test, current_bounds[0]) & torch.lt(x_test, current_bounds[1])
        current_x_test = x_test[current_x_test_mask]
        mean_test = current_model.posterior(current_x_test.unsqueeze(-1)).mean.detach().numpy()
        ax.plot(current_x_test.numpy(), mean_test, 'b-', alpha=0.9, label='surrogate mean')
        var_test = current_model.posterior(current_x_test.unsqueeze(-1)).variance.detach().numpy()
        sd_test = np.sqrt(var_test)
        upper_test = mean_test + 2.0 * sd_test
        lower_test = mean_test - 2.0 * sd_test
        ax.fill_between(current_x_test.numpy(), lower_test.squeeze(), upper_test.squeeze(), color='b', alpha=0.3,
                        label=r'surrogate 2$\sigma$')
    # training points
    ax.plot(x_search.numpy()[0:10], y_search[0:10].numpy(), '.', color='tab:orange', label='initial data')
    ax.plot(x_search.numpy()[10:], y_search[10:].numpy(), '.', color='g', label='bo data')
    # partitions
    for n, b in zip(nodes, bnds):
        ax.plot([b[0].numpy(), b[0].numpy()], [-0.05, 1.45], 'k-')
    # ax.legend()
    ax.set_ylim([-0.02, 1.02])
    ax.set_title('PIONEER_rep{}'.format(rep))
    # ax.set_xlim([0.915, 0.925])
    plt.savefig(datasavedir + '/'+'1d_results_local_models_rep{}'.format(rep)+'.png')


def plot_current_tree(global_search: MultimodalExtremaSearch, rep):
    """Save a plot of the tree structure after the search"""
    # tree figure
    T = global_search.global_state.partition_graph
    pos = nx.nx_agraph.graphviz_layout(T, prog="dot")
    # bounds to put on nodes on plot
    all_nodes, all_bounds = get_bounds_all(global_search.global_state.partition_graph)
    # positioning of bound text
    pos_attrs = {}
    for node, coords in pos.items():
        pos_attrs[node] = (coords[0]-15, coords[1])
    custom_node_attrs = {}
    for node, attrs in zip(all_nodes, all_bounds):
        custom_node_attrs[node] = str(attrs.numpy().round(decimals=3))
    # plot commands
    f, ax = plt.subplots(1, 1, figsize=(12, 8), constrained_layout=True)
    nx.draw(T, pos, with_labels=True, font_weight='bold', ax=ax)
    nx.draw_networkx_labels(T, pos_attrs, labels=custom_node_attrs, ax=ax)
    ax.set_title('PIONEER Partition Tree')
    plt.savefig(datasavedir + '/'+'partition_tree_rep{}'.format(rep)+'.png')

In [ ]:
# experiment functions to calculate metrics
# metrics functions
def count_number_peaks_observed(x_obs, y_obs, num_known_peaks=5):
    """Function to count the number of peaks observed for the mme_noise_jump_1d function"""
    peak1, peak2, peak3, peak4, peak5 = False, False, False, False, False
    for x, y in zip(x_obs, y_obs):
        if 0.003 <= x <= 0.013:
            if y >= 0.95:
                peak1 = True
        elif 0.240 <= x <= 0.250:
            if y >= 0.90:
                peak2 = True
        elif 0.445 <= x <= 0.455:
            if y >= 0.73:
                peak3 = True
        elif 0.673 <= x <= 0.683:
            if y >= 0.47:
                peak4 = True
        elif 0.925 <= x <= 0.935:
            if y >= 0.238:
                peak5 = True
    num_peaks_observed = peak1 + peak2 + peak3 + peak4 + peak5
    return num_peaks_observed


def count_evaluations_for_all_peaks(x_obs, y_obs):
    """Count the number of function evaluations before finding all peaks"""
    i = 0
    num_evals_for_all_peaks = x_obs.shape[0]
    peak1, peak2, peak3, peak4, peak5 = False, False, False, False, False
    for x, y in zip(x_obs, y_obs):
        if 0.003 <= x <= 0.013:
            if y >= 0.95:
                peak1 = True
        elif 0.240 <= x <= 0.250:
            if y >= 0.90:
                peak2 = True
        elif 0.445 <= x <= 0.455:
            if y >= 0.73:
                peak3 = True
        elif 0.673 <= x <= 0.683:
            if y >= 0.47:
                peak4 = True
        elif 0.925 <= x <= 0.935:
            if y >= 0.238:
                peak5 = True
        i += 1
        if peak1 and peak2 and peak3 and peak4 and peak5:
            num_evals_for_all_peaks = i
            break
    return num_evals_for_all_peaks

In [ ]:
# replicated searches

# search and experiment settings
n_replications = 15
global_bounds = torch.tensor([0.0, 1.0], dtype=dtype)
n_evals = 130

# data holders
x_observed_all = []
y_observed_all = []
distinct_peaks = []
function_evaluations = []

# experiment loop
for rep in range(1, n_replications + 1):
    print(f"\nTrial {rep:>2} of {n_replications} ", end="")
    t0 = time.monotonic()
    # initialize search
    current_global_search = MultimodalExtremaSearch(60,global_bounds,outcome_objective,total_iteration_limit=n_evals)
    # run search
    current_global_search.run_global_search()

    # collect data
    x_observed_all.append(current_global_search.global_state.x_global)
    y_observed_all.append(current_global_search.global_state.y_global)

    # run and save plots
    # results with local model
    plot_local_models_1d(current_global_search.global_state.partition_graph,
                         current_global_search.global_state.x_global,
                         current_global_search.global_state.y_global,
                         rep)
    # partitioning results tree
    plot_current_tree(current_global_search, rep)

    # calculate metrics
    num_peaks_observed = count_number_peaks_observed(current_global_search.global_state.x_global,
                                                     current_global_search.global_state.y_global)
    print("Observed {num:} peaks".format(num=num_peaks_observed),)
    distinct_peaks.append(num_peaks_observed)
    fe = count_evaluations_for_all_peaks(current_global_search.global_state.x_global,
                                         current_global_search.global_state.y_global)
    print("Function evaluations to find all peaks: {}".format(fe))
    function_evaluations.append(fe)
    # time for search
    t1 = time.monotonic()
    print("Time to complete search: {}".format(t1-t0))

In [ ]:
# calculate summary metrics
# peak ratio
num_known_peaks = 5
# n_replications = 15
peak_ratio = sum(distinct_peaks) / (num_known_peaks * n_replications)
print("Peak ratio: {}".format(peak_ratio))
# success rate
num_successes = 0
for i in range(n_replications):
    if distinct_peaks[i] == num_known_peaks:
        num_successes += 1
success_ratio = num_successes / n_replications
print("Success ratio: {}".format(success_ratio))
print("Average function evaluations: {}".format(sum(function_evaluations)/n_replications))

In [ ]:
# save data to file for potential future use
methods_list = ['PIONEER']
pr_list = [peak_ratio]
sr_list = [success_ratio]
fe_list = [function_evaluations]
import pickle
with open(datasavedir + '/' + 'mme_metrics.pkl', 'wb') as f:
    pickle.dump([methods_list, pr_list, sr_list, fe_list], f)
# to retrieve
# with open(datasavedir + '/' + 'mme_metrics.pkl') as f:
#     methods_list, pr_list, sr_list, fe_list = pickle.load(f)